In [1]:
# importing required modules
import pandas as pd 
import mysql.connector as msql
import math
import os
import re

In [2]:
# Importing Excel file form directory 
directory_path =(r'C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\TSWRDC Code\Source\New Student Info')

excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

for file in excel_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_excel(file_path)
    print(f"Data from {file}:")

Data from TS_TT_General Info.xlsx:


In [3]:
#Reading She for STEM Incubator file present on source files
directory_path =(r"C:\Users\spjay\OneDrive - VigyanShaala\02 Products  Initiatives\01 Kalpana\05 Kalpana M&E\00 DBMS 1.0\Kalpana\Kalpana\04 Incubator - Curriculum Cohort\TSWRDC Code\Source\Course Data")
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    data = pd.read_csv(file_path)
    print(f"Data from {file}:")

Data from TSWRDC - Feb - Apr 2024-1724831698103.csv:


In [4]:
# Define a list of column names to extract for putting into database 
columns_to_extract = ['Name', 'Email', 'Segment', 'Mobile', 'Enroll Date', 'Assigned Through', 'Course Name','Start Date','Last Login']

# Use the loc method to extract the specified columns
data = data.loc[:, columns_to_extract]

In [7]:
# Few rows of data
data.shape

(1087, 9)

In [7]:
# Function to clean email addresses
def clean_email(email):

    # Convert to lowercase and remove extra spaces
    cleaned_email = email.lower().strip()
    # Remove patterns like ".com.1"
    cleaned_email = re.sub(r'\.com\.\d+', '.com', cleaned_email)
    return cleaned_email

# Apply the function to the 'email' column
df['Email'] = df['Email'].apply(clean_email)

In [8]:
# Column Names of data
df.columns

Index(['Email', 'Name', 'Phone', 'Gender', 'Currently Pursuing Degree',
       'Subject Area', 'Name of College', 'Country', 'State/Union Territory',
       'District', 'City Category', 'Caste/Category', 'Annual Income',
       'Assigned through', 'Incubator Course Name', 'Incubator Batch'],
      dtype='object')

In [9]:
# Renaming Column name in data
df.columns =['Email', 'Name', 'Phone', 'Gender', 'Currently_Pursuing_Degree',
       'Subject_Area', 'Name_of_College_University', 'Country',
       'State_Union_Territory', 'District', 'City_Category', 'Caste_Category',
       'Annual_Family_Income', 'Assigned_Through','Incubator_Course_Name','Incubator_Batch']

In [10]:
df.columns

Index(['Email', 'Name', 'Phone', 'Gender', 'Currently_Pursuing_Degree',
       'Subject_Area', 'Name_of_College_University', 'Country',
       'State_Union_Territory', 'District', 'City_Category', 'Caste_Category',
       'Annual_Family_Income', 'Assigned_Through', 'Incubator_Course_Name',
       'Incubator_Batch'],
      dtype='object')

# General Information MySQL table

In [ ]:
# Connecting to MySQL Database
conn= msql.connect(host='',user='',password="",database="",auth_plugin='')


In [12]:
cursor =conn.cursor()

In [13]:
for i, row in df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    columns = ','.join(df.columns)
    placeholders = ','.join(['%s']*len(row))
    # Construct the INSERT query with ON DUPLICATE KEY UPDATE clause
    query = f"INSERT INTO she_for_stem.01_general_information ({columns}) VALUES ({placeholders}) ON DUPLICATE KEY UPDATE "
    query += ", ".join([f"{col}=VALUES({col})" for col in df.columns if col != 'Email'])
    # Execute the query
    cursor.execute(query, tuple(row))

In [14]:
# Inserting records from data file, if already present ignoring it.
for i,row in df.iterrows():
    row = [None if isinstance(val, float) and math.isnan(val) else val for val in row] # replace "nan" values with None
    cursor.execute("INSERT IGNORE INTO she_for_stem.01_general_information(Email, Name, Phone, Gender, Currently_Pursuing_Degree, Subject_Area, Name_of_College_University, Country, State_Union_Territory, District, City_Category, Caste_Category, Annual_Family_Income, Assigned_Through,Incubator_Course_Name,Incubator_Batch) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tuple(row))


In [15]:
conn.commit()

# Payment details MySQL table

In [16]:
# Creating new table which for Payment which is taken from incubator graphy sheet
Payment=pd.DataFrame(df[["Email","Assigned_Through"]])

In [17]:
# Assigning fee cost to different cathegory of enrollment
Payment["Assigned_Through"]=Payment["Assigned_Through"].replace(['Scholarship'],'0')

In [18]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [19]:
# Inserting data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("insert Ignore into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [20]:
# Replacing data with new data into 02_payment_details 
for i,row in Payment.iterrows():
    cursor.execute("REPLACE into 02_payment_details (Email,Incubator_Fee) values(%s,%s)",tuple(row))

In [21]:
conn.commit()

# Enroll Date MySQL Table

In [22]:
# Creating new table which for Enroll_Dates which is taken from incubator graphy sheet
Enroll=pd.DataFrame(data[["Email", 'Enroll Date']])

In [23]:
# Extracting only Enroll_Date
Enroll[['Enroll Date','Time']]=Enroll['Enroll Date'].str.split(' ',expand=True)
Enroll=Enroll.drop(["Time"],axis=1)

In [24]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [25]:
# Inserting data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("insert IGNORE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [26]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Enroll.iterrows():
    cursor.execute("REPLACE into 03_enroll_date (Email,Incubator) values(%s,%s)",tuple(row))

In [27]:
conn.commit()

# Batch Lastlogin Startdate MySQL Table

In [8]:
# Creating new table which for Batch which is taken from incubator graphy sheet
Batch=pd.DataFrame(data[["Email", 'Last Login','Start Date']])

In [12]:
Batch

,Email,Last Login,Start Date
0,vijayacads@gmail.com,04-11-2023,30-09-2023
1,sreejith.sreenivasan@vigyanshaala.com,06-11-2023,21-09-2023
2,monikadaw5353@gmail.com,02-02-2023,Not Started
3,prachijagdale0123@gmail.com,30-09-2023,Not Started
4,nikitasingh4615@gmail.com,16-10-2023,07-10-2023
...,...,...,...
1082,tambolisamina5@gmail.com,-,Not Started
1083,anushkamarkad.27@gmail.com,-,Not Started
1084,durvajondhale.work@gmail.com,-,Not Started
1085,unnatibansod3201@gmail.com,-,Not Started


In [10]:
# Extracting only Enroll_Date
Batch[['Last Login','Time']]=Batch['Last Login'].str.split(' ',expand=True)
Batch=Batch.drop(["Time"],axis=1)

In [11]:
# Function to convert 'Start Date' format
def convert_start_date(date_str):
    if re.match(r'\d{4}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%Y/%m/%d').strftime('%d-%m-%Y')
    return date_str

# Function to convert 'Last Login' format
def convert_last_login(date_str):
    if re.match(r'\d{2}/\d{2}/\d{2}', date_str):
        return pd.to_datetime(date_str, format='%d/%m/%y').strftime('%d-%m-%Y')
    return date_str


# Apply the conversion functions to the respective columns
Batch['Start Date'] = Batch['Start Date'].apply(convert_start_date)
Batch['Last Login'] = Batch['Last Login'].apply(convert_last_login)

In [13]:
# Connecting to MySQL Database
conn= msql.connect(host='localhost',user='root',password="VS@123",database="she_for_stem",auth_plugin='mysql_native_password')
cursor =conn.cursor()

In [15]:
# Inserting data to 15_batch_lastlogin_startdate table
for i,row in Batch.iterrows():
    cursor.execute("insert IGNORE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [16]:
# Replacing data  with new data to 03_enroll_date table
for i,row in Batch.iterrows():
    cursor.execute("REPLACE into 15_batch_lastlogin_startdate (Email,Incubator_Last_Login,Incubator_Start_Date) values(%s,%s,%s)",tuple(row))

In [17]:
conn.commit()